In [1]:
import sys
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/')
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = '/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/'

In [ ]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')

In [2]:
meteo = dict()
    
for a in list(range(1988, 2026)):
    meteo[a] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/monthly_weather_forecast/{a}.geojson')

---

In [ ]:
from tqdm import tqdm

In [ ]:
import pickle 
with open('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/data_for_weather_kriging.pkl', 'rb') as f:
     loaded_dict = pickle.load(f)

In [ ]:
loaded_dict = dict()

month_dic = {m: n+1 for n, m in enumerate(['gennaio', 'febbraio', 'marzo', 'aprile', 'maggio', 'giugno', 'luglio', 'agosto', 'settembre', 'ottobre', 'novembre', 'dicembre'])}

for y, gdf in tqdm(meteo.items()):

    loaded_dict[y] = {}

    aux=gdf.copy()
    aux.drop(columns=['id_parametro', 'id_aggregazione', 'Denominazione', 'anno'], inplace=True)
    aux.rename(columns={'Quota (m s.l.m.)': 'height'}, inplace=True)
    aux.to_crs(epsg=3857, inplace=True)

    for param, data in tqdm(aux.groupby('parametro_calcolato')):
        new_gdf = pd.melt(
                data,
                id_vars=['height', 'geometry'],
                value_vars = ['gennaio', 'febbraio', 'marzo', 'aprile', 'maggio', 'giugno', 'luglio', 'agosto', 'settembre', 'ottobre', 'novembre', 'dicembre'], 
                var_name='month',
                value_name=param,
                ignore_index=True)
        new_gdf.loc[: ,'month'] = new_gdf.loc[: ,'month'].map(month_dic)
        loaded_dict[y][param] = new_gdf

In [ ]:
import pickle

with open('/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/data_for_weather_kriging.pkl', 'wb') as f:
    pickle.dump(loaded_dict, f)

---

In [11]:
for y in meteo.keys():
    if meteo[y].loc[:, 'gennaio':'dicembre'].dtypes.unique().shape[0] > 1:
        print(f'Year {y} has dtypes: ', meteo[y].loc[:, 'gennaio':'dicembre'].dtypes.unique())
    elif (meteo[y].loc[:, 'gennaio':'dicembre'].dtypes.unique().shape[0] == 1) and (meteo[y].loc[:, 'gennaio':'dicembre'].dtypes.unique()[0] != 'float64'):
            print(f'Year {y} has unique dtype: ', meteo[y].loc[:, 'gennaio':'dicembre'].dtypes.unique())

Year 2025 has dtypes:  [dtype('float64') dtype('O')]


Only 2025 has object dtype in some variable, but I won't predict on that year